# Bio 302 Homework 6

There are two parts to this homework, some problems given in this notebook and some problems from the textbook. Note that some sections of Chapter 5, and all of Chapter 6, is strongly-suggested background reading.

**Part 1: Notebook**

Complete the python/exploration problems in this notebook. Please solve them in the indicated space, and ensure that your solutions pass all supplied tests.

**Part 2: Textbook**

Read sections of Chapter 5 pertaining to PSI-BLAST. Feel free to look over the remaining method.

Read Chapter 6, on multiple sequence alignment.

Chapter 6 Discussion Questions: 6-1; Problems/Computer Lab: 6-1, 6-2

Please use the *FOXP2* sequence data you'll assemble for Problem 3 in this notebook as the input for the computer lab exercises 6-1 and 6-2 above. Two birds.

**Submission**

Submit your solutions as a link to a Gist via a Slack DM.

In [ ]:
# -- Some standard imports for the plotting package matplotlib,
#    and some setup. You'll want to leave these as is.
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [12.0, 8.0]

## Problem 1

Run a PSI-BLAST at NCBI on the protein sequence
included in "mystery_seq.fasta." Use Expect threshold
= 0.1, Word size = 2, and leave all other parameters
at their defaults. Use database = 'nr'.

(a) How many hits have an initial e-value of 1e-5 or
    smaller in the first (blastp) iteration?

(b) Use the default inclusion cutoff, and run
    iteration 2.  How many new proteins could be found
    with an evalue better than 1e-5 on this iteration?
    How many iterations do you have to repeat until no
    new sequences can be found? (If it does not
    converge after 6 iterations, stop.)

(c) Click on the "Distance tree of results" at the
    bottom of the page to see a phylogenetic
    representation of all the hits found.  Is this
    protein unique to this species or just
    Crenarchaea, just Archaea, just Archaea &
    Bacteria, or is it found in all three domains of
    life?

(d) Based on the e-value and percent identity against
    the top scoring hit of newly-found sequences, do
    you think these hit sequences are orthologs (same
    function), paralogs (related function), or have
    unrelated function?

[Solution to Problem 1 here, or in separate document.]

## Problem 2

Write a function called `count_msa_gaps`, that
invokes the program `muscle` to compute a multiple
sequence alignment.

I've included muscle executables for both mac and windows in the repository, for convenience. They come from 

 http://www.drive5.com/muscle/downloads.htm

Several useful functions are provided to you, below. You should study them and incorporate them into your solutions, because that will save you a bunch of time.

- `run_command` This runs a system command that you specify as a list of strings. See examples.
- `get_os` This will tell you what kind of system you're on. (Alternatively, just use the muscle binary appropriate to your system.) See examples.
- `read_fasta_text` This you've seen before. The one difference is that you pass it fasta text as an argument, instead of a filename.

There are a couple examples of `muscle` being invoked with `run_command`. Its output gets collected into a dictionary with keys `output` and `error`. The former is for normal program output, typically the actual MSA. The latter is for genuine errors, or more often just for progress messages that you wouldn't want cluttering the output.

Your function `count_msa_gaps` should take one
argument, the name of a fasta file. It should invoke muscle with
no extra arguments, capture its output, and print the
proportion of each sequence that was gaps.

Here is an example run of `count_msa_gaps` over the provided sample file, `actn3.fasta`. Your function should produce the same output.

```
> count_msa_gaps('actn3.fasta')
gi|66365807|gb|AAH96323.1| ACTN3 protein [Homo sapiens],0.31
gi|183985943|gb|AAI66600.1| Actn3 protein [Rattus norvegicus],0.00
gi|7304855|ref|NP_038484.1| alpha-actinin-3 [Mus musculus],0.00
```

That is, muscle did not add any gaps to the mouse or
rat ACTN3 protein sequences, but fully 31% of the
multiply-aligned human sequence was gaps. Makes sense;
the human sequence is way longer than the mouse/rat
sequences!

In [ ]:
import subprocess
def run_command(args):
    "Run a command-line, provided as a list of args, and return the result."
    proc = subprocess.Popen(args,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)
    stdout, stderr = map(lambda x: x.strip().decode('utf-8'), proc.communicate())
    return dict(
        output=stdout,
        error=stderr,
    )

In [ ]:
import sys
def get_os():
    "Returns 'mac_os_x' if you're on a mac, 'windows' if on windows."
    my_os = sys.platform
    if my_os == 'darwin':
        return 'mac_os_x'
    elif my_os == 'win32':
        return 'windows'
    else:
        return None

Here's an example of using `run_command` to invoke `muscle`. It shows what happens if you call `muscle` without any additional arguments:

In [ ]:
my_os = get_os()

if my_os == 'mac_os_x':
    result = run_command('./muscle3.8.31_i86darwin64')
else:
    result = run_command('./muscle3.8.31_i86win32.exe')
print("[OUTPUT]\n", result['output'])
print("[ERROR]\n", result['error'])

This example shows how to provide `muscle` with arguments. Note that the list we pass in to `run_command` is a regular python list, so you're free to use variables, string operations, list operations, all the usual python stuff in constructing such lists.

You may wish to consult the `muscle` manual to see all the arguments it accepts. Here are the main options:

http://www.drive5.com/muscle/manual/options.html

And here is the exhaustive list of all the tuning options muscle knows about:

http://www.drive5.com/muscle/manual/valueopts.html

In [ ]:
# this amounts to running "./muscle3.8.31_i86darwin64 -in test.fasta" in a system terminal
if my_os == 'mac_os_x':
    result = run_command(['./muscle3.8.31_i86darwin64', '-in', 'test.fasta'])
else:
    result = run_command(['./muscle3.8.31_i86win32.exe', '-in', 'test.fasta'])
print("[OUTPUT]\n", result['output'])
print("[ERROR]\n", result['error'])

In [ ]:
def read_fasta_text(fasta_text):
    'Returns a list of dictionaries of header/sequence pairs from FASTA-formatted text.'
    out = []
    for seqrec in fasta_text.split('>'):
        if not seqrec:
            continue
        lines = seqrec.split('\n')
        header = lines[0]
        seq = ''.join(lines[1:])
        out.append(dict(header=header, sequence=seq))
    return out

In [ ]:
# -- Solution to Problem 2 here.

## Problem 3

Re-use your function from Problem 2 to study the
effect of gap cost on multiple sequence alignments.

Grab the *protein* sequences derived from the gene
*FOXP2* for human (Homo sapiens), chimp (Pan
troglodytes), mouse (Mus musculus), cow (Bos taurus),
and African clawed frog (Xenopus laevis).  Each
sequence should be about 700 amino acids
long. Further, attempt to grab proteins named
something like "FOXP2 forkhead box P2 [ Homo sapiens
(human) ]"; ie, avoid the ones that have 'isoform' or
'splice variant' or other qualifiers in the title. Put
all the sequences into a single fasta file called
`foxp2_proteins.fasta`, and include it in your
submission.

The program should set muscle's `gapopen` cost
in turn to -0.1, -1.0, -10.0, and -100.0. The
`gapextend` cost should always be -1.0. For each value
of `gapopen`, compute the proportion of the entire
alignment that was gaps. Use matplotlib to plot this
proportion as a function of the muscle `gapopen` cost,
and be sure the plot appears in this notebook. Label
the plot appropriately.

You may wish to consult muscle's documentation, linked above.


Discuss the plot that you've produced. In particular:

1. Describe and interpret the
pattern of dependence that you observe.
2. Does the pattern you see make sense to you?
3. Is the dependence on gapopen cost stronger or weaker than you'd expect?

In [ ]:
# Solution to Problem 3 here.